In [72]:
import pandas as pd
import numpy as np

from numpy import ndarray

In [73]:
def sigmoid(m):
    return 1/(1+np.exp(-m))

def MSE(y: ndarray, y_pred: ndarray) -> float:
    return np.power((y - y_pred), 2).mean()

def norm(a, b):
    return np.linalg.norm((a-b))

def cholesky(matrix:ndarray):
    R = np.zeros_like(matrix)
    n = matrix.shape[0]
    for j in range(n):
        for i in range(j,n):
            if i == j:
                R[i,j] = np.sqrt(matrix[i,j] - np.sum(R[i,:]**2))
            else:
                R[i,j] = (matrix[i,j] - np.sum( R[i,:] * R[j,:] )) / R[j,j]
    return R

def backwardSub_upper(U, b):
    out = np.zeros((b.shape[0],U.shape[0]))

    for c in range(b.shape[0]):
        out[c,0] = b[c,0] / U[0,0]
        for i in range(1, U.shape[0]):
            acc = 0
            for j in range(i):
                acc += U[j,i] * out[c,j]
            out[c,i] = (b[c,i] - acc) / U[i,i]
    return out

def backwardSub_lower(L, b):
    out = np.zeros((b.shape[0],L.shape[0]))

    for c in range(b.shape[0]):
        out[c,-1] = b[c,-1] / L[-1,-1]
        
        for i in range(L.shape[0] - 1, -1, -1):
            acc = 0
            for j in range(i+1, L.shape[0]):
                acc += L[j,i] * out[c,j]
                
            out[c,i] = (b[c,i] - acc) / L[i,i]
    return out

In [74]:
tr  = pd.read_csv("datasets/ML-CUP21-TR.csv", header=None, index_col=0)
ts  = pd.read_csv("datasets/ML-CUP21-TS.csv", header=None, index_col=0)
tr_x,tr_y = tr.iloc[:,:10].to_numpy(), tr.iloc[:,10:].to_numpy()
# ts_x,ts_y = ts.iloc[:,:10].to_numpy().T, ts.iloc[:,10:].to_numpy()

#We have training set tr_x 1182 rows and 10 columns
#We have training set tr_y 1182 rows and 2 columns


In [75]:
# R = np.linalg.cholesky(tr_x.T @ tr_x).T
# b = tr_y.T @ tr_x
# z = backwardSub_upper(R,b)
# w = backwardSub_lower(R.T,z)
# MSE(w @ tr_x.T, tr_y.T)

In [117]:
class ENeuralN:
    
    def __init__(self,features:int, hidden:int, alfa_reg:float):
        
        self.W1 = np.random.rand(hidden, features)
        self.alfa_reg = alfa_reg
        self.W2 = None
        
    def fit_NE(self, x:ndarray, y:ndarray):
        h =  sigmoid(self.W1 @ x.T).T
        dim = h.shape[1]
        self.W2 =  np.linalg.inv(h.T @ h + self.alfa_reg * np.eye(dim)) @ h.T @ y 
        return

    def fit_chol(self, x:ndarray, y:ndarray):
        h =  sigmoid(self.W1 @ x.T).T
        dim = h.shape[1]        
        R = np.linalg.cholesky(h.T @ h  + self.alfa_reg * np.eye(dim)).T
        z = backwardSub_upper(R, y.T @ h)
        self.W2 = backwardSub_lower(R.T,z)
        return
    
    def __call__(self, x:ndarray):
        h =  sigmoid(self.W1 @ x.T).T
        o = (self.W2 @ h.T).T
        return o

In [123]:
model = ENeuralN(10, 100, 1e-05)

model.fit_chol(tr_x, tr_y)
out = model(tr_x)

# print(norm(out, tr_y))
print(MSE(tr_y, out))

1.853343567406436
